In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.stattools import coint


In [2]:
# Get the data from tsm and qcom with inteval of 30 minutes

tsm = yf.Ticker("ADBE")
qcom = yf.Ticker("MSFT")

tsm_data = tsm.history(period="1y", interval="1h")
qcom_data = qcom.history(period="1y", interval="1h")

tsm_data.reset_index(inplace=True)
qcom_data.reset_index(inplace=True)

if ('Date' in tsm_data.columns) and ('Date' in qcom_data.columns):
    print("Date column exists")
    tsm_data.rename(columns={"Date": "Datetime"}, inplace=True)
    qcom_data.rename(columns={"Date": "Datetime"}, inplace=True)

tsm_data.set_index("Datetime", inplace=True)
qcom_data.set_index("Datetime", inplace=True)

In [3]:
# plot both tickers on go figure

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'], name='TSM'))
fig.add_trace(go.Scatter(x=qcom_data.index, y=qcom_data['Close'], name='QCOM'))

fig.show()

In [4]:
print(tsm_data)

                                 Open        High         Low       Close  \
Datetime                                                                    
2023-06-05 09:30:00-04:00  431.290009  433.375000  426.959991  428.579987   
2023-06-05 10:30:00-04:00  428.704987  434.489990  428.459991  434.165009   
2023-06-05 11:30:00-04:00  434.059998  439.510590  434.040009  438.980011   
2023-06-05 12:30:00-04:00  438.920013  440.379913  438.454987  439.825012   
2023-06-05 13:30:00-04:00  439.869995  439.869995  434.285004  434.820007   
...                               ...         ...         ...         ...   
2024-06-04 11:30:00-04:00  445.989990  445.989990  443.785004  445.100006   
2024-06-04 12:30:00-04:00  445.149994  445.744995  443.684998  445.380005   
2024-06-04 13:30:00-04:00  445.545013  447.299988  444.549988  447.195007   
2024-06-04 14:30:00-04:00  447.350006  449.399994  446.220001  447.059998   
2024-06-04 15:30:00-04:00  447.130005  448.970001  446.600006  448.369995   

In [5]:
# plot eith candle stick

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

# plot moving average of 20 ticks and 80 ticks

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

fig.show()

In [6]:
# count the number of times the 5 ticks MA crosses the 20 ticks MA

tsm_data['5T_MA'] = tsm_data['Close'].rolling(window=5).mean()
tsm_data['20T_MA'] = tsm_data['Close'].rolling(window=20).mean()

tsm_data['signal'] = np.where(tsm_data['5T_MA'] > tsm_data['20T_MA'], 1, 0)
tsm_data['signal'] = tsm_data['signal'].diff()


print(tsm_data['signal'].value_counts())

# plot the buy and sell signals

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

buy_signals = tsm_data[tsm_data['signal'] == 1]
sell_signals = tsm_data[tsm_data['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))

fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

signal
 0.0    1652
 1.0      52
-1.0      51
Name: count, dtype: int64


In [7]:
# calculate profit and loss

tsm_data['buy_price'] = np.where(tsm_data['signal'] == 1, tsm_data['Close'], np.nan)
tsm_data['sell_price'] = np.where(tsm_data['signal'] == -1, tsm_data['Close'], np.nan)

tsm_data['buy_price'] = tsm_data['buy_price'].ffill()
tsm_data['sell_price'] = tsm_data['sell_price'].ffill()

tsm_data['PnL'] = tsm_data['sell_price'] - tsm_data['buy_price']

print(tsm_data['PnL'].sum())

# plot the profit and loss

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

buy_signals = tsm_data[tsm_data['signal'] == 1]
sell_signals = tsm_data[tsm_data['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

1781.3962707519531


In [8]:
ratio_0 = (tsm_data['Close'].iloc[0]) / (qcom_data['Close'].iloc[0])
ratio_0

1.2740568028458759

In [9]:
qcom_data['Close_NORM'] = qcom_data['Close'].multiply(ratio_0).round(6)

In [10]:
tsm_qcom_dif = tsm_data['Close'] - qcom_data['Close_NORM']
tsm_qcom_dif

Datetime
2023-06-05 09:30:00-04:00   -4.277344e-07
2023-06-05 10:30:00-04:00    4.068931e+00
2023-06-05 11:30:00-04:00    8.485012e+00
2023-06-05 12:30:00-04:00    9.620611e+00
2023-06-05 13:30:00-04:00    5.118883e+00
                                 ...     
2024-06-04 11:30:00-04:00   -8.032089e+01
2024-06-04 12:30:00-04:00   -8.140426e+01
2024-06-04 13:30:00-04:00   -8.180661e+01
2024-06-04 14:30:00-04:00   -8.074350e+01
2024-06-04 15:30:00-04:00   -8.172683e+01
Length: 1756, dtype: float64

In [11]:
# Plot the difference between the two tickers

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif, name='TSM-QCOM'))

fig.show()

In [12]:
# Plot the two normalized tickers

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'], name='TSM'))
fig.add_trace(go.Scatter(x=qcom_data.index, y=qcom_data['Close_NORM'], name='QCOM'))

fig.show()

In [13]:
# Plot the difference and its moving average

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif, name='TSM-QCOM'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=20).mean(), name='20 Ticks MA'))

fig.show()


In [14]:
tsm_qcom_dif = tsm_qcom_dif.to_frame()
tsm_qcom_dif.columns = ['Close']

In [15]:
tsm_qcom_dif['10T_MA'] = tsm_qcom_dif['Close'].rolling(window=10).mean()
tsm_qcom_dif['60T_MA'] = tsm_qcom_dif['Close'].rolling(window=60).mean()

tsm_qcom_dif['signal'] = np.where(tsm_qcom_dif['10T_MA'] > tsm_qcom_dif['60T_MA'], -1, 0)
tsm_qcom_dif['signal'] = tsm_qcom_dif['signal'].diff()

print(tsm_qcom_dif['signal'].value_counts())

# Plot the signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif['Close'], name='TSM-QCOM'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=10).mean()['Close'], name='10 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=60).mean()['Close'], name='60 Ticks MA'))

buy_signals = tsm_qcom_dif[tsm_qcom_dif['signal'] == 1]
sell_signals = tsm_qcom_dif[tsm_qcom_dif['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

 # -1 buy || 1 sell

tsm_data['signal'] = tsm_qcom_dif['signal']
qcom_data['signal'] = -tsm_qcom_dif['signal'] # opposite signal

fig.update_layout(title='TSM-QCOM Moving Average Crossover Strategy (Short Average = 10, Long Average = 60) (60 days)')

fig.show()

# plot the profit and loss

tsm_data['buy_price'] = np.where(tsm_data['signal'] == 1, tsm_data['Close'], np.nan)
tsm_data['sell_price'] = np.where(tsm_data['signal'] == -1, tsm_data['Close'], np.nan)

qcom_data['buy_price'] = np.where(qcom_data['signal'] == 1, qcom_data['Close_NORM'], np.nan)
qcom_data['sell_price'] = np.where(qcom_data['signal'] == -1, qcom_data['Close_NORM'], np.nan)

tsm_profit = tsm_data['sell_price'].sum() - tsm_data['buy_price'].sum()
qcom_profit = qcom_data['sell_price'].sum() - qcom_data['buy_price'].sum()

print('TSMC Profit => ', tsm_profit)
print('QCOM Profit => ', qcom_profit)

pair_tsm_qcom_profit = tsm_profit + qcom_profit

print('Pair profit => ', pair_tsm_qcom_profit)


signal
 0.0    1715
-1.0      20
 1.0      20
Name: count, dtype: int64


TSMC Profit =>  -46.767242431640625
QCOM Profit =>  62.40572699999939
Pair profit =>  15.638484568358763


In [16]:
#Consider TSM as the first and QCOM as the second ticker.
#When there is a buy signal for TSM, we buy TSM and sell QCOM.
#When there is a sell signal for TSM, we sell TSM and buy QCOM.

In [17]:
#Copy signal column from difference dataframe to TSM dataframe
tsm_data['signal'] = tsm_qcom_dif['signal']
qcom_data['signal'] = -tsm_qcom_dif['signal']

In [18]:
tsm_data.reset_index(inplace=True)
qcom_data.reset_index(inplace=True)

# Convert datetime columns to indexes
tsm_data.set_index('Datetime', inplace=True)
qcom_data.set_index('Datetime', inplace=True)

# Find common datetime values
common_dates = tsm_data.index.intersection(qcom_data.index)

# Align the data and handle missing values
tsm_close_aligned = tsm_data.loc[common_dates, 'Close']
qcom_close_aligned = qcom_data.loc[common_dates, 'Close_NORM']

# Perform cointegration test
cointegration_test_result = coint(tsm_close_aligned, qcom_close_aligned)


# Print the result
print("Cointegration Test Result:")
print("Test statistic:", cointegration_test_result[0])
print("p-value:", cointegration_test_result[1])
print("Critical values:", cointegration_test_result[2])



Cointegration Test Result:
Test statistic: -1.8237753610390068
p-value: 0.6181032836801506
Critical values: [-3.90269128 -3.33961375 -3.04686752]


In [19]:
# Calculate the z score of the difference
tsm_qcom_dif['z_score'] = (tsm_qcom_dif['Close'] - tsm_qcom_dif['Close'].rolling(window=20).mean()) / tsm_qcom_dif['Close'].rolling(window=20).std()

In [20]:
# function to implement a flag that is (-1 sell order open, 0 no order open, 1 buy order open) to control the signal_z_score signals

def restrict_signals(dataset, signal_col, flag_col):
    dataset[flag_col] = 0
    # cicle to put flags
    for i in range(dataset.shape[0]):
        if dataset[signal_col].iloc[i] == 1:
            if dataset[flag_col].iloc[i] < 1:
                dataset[flag_col].iloc[i] += 1
                print('Buy signal')
            else:
                dataset[signal_col].iloc[i] = 0
                print('Reset buy signal')
        elif dataset[signal_col].iloc[i] == -1:
            if dataset[flag_col].iloc[i] > -1:
                dataset[flag_col].iloc[i] += -1
                print('Sell signal')
            else:
                dataset[signal_col].iloc[i] = 0
                print('Reset sell signal')

    return dataset
        

In [21]:
# plot the z score along with the buy and sell signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif['z_score'], name='Z Score'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=[2] * len(tsm_qcom_dif), name='Upper Threshold'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=[-2] * len(tsm_qcom_dif), name='Lower Threshold'))

#buy_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['z_score'] < -2]
#sell_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['z_score'] > 2]

# TODO: check this
#tsm_qcom_dif['signal_z_score'] = np.where(((tsm_qcom_dif['z_score'] < -2) and (tsm_qcom_dif['z_score_sig_flag'] < 1)), 1, 0) + np.where(((tsm_qcom_dif['z_score'] > 2) and (tsm_qcom_dif['z_score_sig_flag'] > 1)), -1, 0)
tsm_qcom_dif['signal_z_score'] = np.where((tsm_qcom_dif['z_score'] < -2), 1, 0) + np.where((tsm_qcom_dif['z_score'] > 2), -1, 0)

#tsm_qcom_dif = restrict_signals(tsm_qcom_dif, 'signal_z_score', 'z_score_sig_flag')

""" INIT """
flag_signal_z_score = 0

for i in range(tsm_qcom_dif.shape[0]):
    print(tsm_qcom_dif['signal_z_score'].iloc[i])
    if tsm_qcom_dif['signal_z_score'].iloc[i] == 1:
        print('in', flag_signal_z_score)
        if flag_signal_z_score < 1:
            flag_signal_z_score = 1
            print('Buy signal')
        else:
            tsm_qcom_dif['signal_z_score'].iloc[i] = 0
            print('Reset buy signal')
    elif tsm_qcom_dif['signal_z_score'].iloc[i] == -1:
        if flag_signal_z_score > -1:
            flag_signal_z_score = -1
            print('Sell signal')
        else:
            tsm_qcom_dif['signal_z_score'].iloc[i] = 0
            print('Reset sell signal')
    print(tsm_qcom_dif['signal_z_score'].iloc[i])
    

""" END """
# plot the signals
buy_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == 1]
sell_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == -1]

fig.add_trace(go.Scatter(x=buy_signals_z_score.index, y=buy_signals_z_score['z_score'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals_z_score.index, y=sell_signals_z_score['z_score'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.update_layout(title='Z Score with Buy/Sell Signals (60 days)')

fig.show()


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
Sell signal
-1
-1
Reset sell signal
0
-1
Reset sell signal
0
0
0
0
0
0
0
0
0
-1
Reset sell signal
0
-1
Reset sell signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
in -1
Buy signal
1
0
0
0
0
0
0
0
0
0
0
0
0
-1
Sell signal
-1
-1
Reset sell signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
in -1
Buy signal
1
1
in 1
Reset buy signal
0
1
in 1
Reset buy signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
in 1
Reset buy signal
0
0
0
0
0
1
in 1
Reset buy signal
0
0
0
0
0
0
0
0
0
0
0
0
0
1
in 1
Reset buy signal
0
1
in 1
Reset buy signal
0
1
in 1
Reset buy signal
0
1
in 1
Reset buy signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
Sell signal
-1
-1
Reset sell signal


/tmp/ipykernel_239381/2492969377.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_239381/2492969377.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_239381/2492969377.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_239381/2492969377.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

### Check if the number of sell orders is equal to buy orders and remove the last one in case of odd number of orders.


In [22]:

count_buys = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == 1].shape[0]
count_sells = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == -1].shape[0]

print('Count buys => ', count_buys)
print('Count sells => ', count_sells)

dif_buys_sells = count_buys - count_sells
print('Dif buys/sells => ', dif_buys_sells)

tsm_qcom_dif.reset_index(inplace=True)

if dif_buys_sells > 0:
    last_buy_signal = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == 1].iloc[[-1]]['Datetime']
    #print(last_buy_signal)
    if not last_buy_signal.empty:
        print('Datetime of signal to set to 0 => ', str(last_buy_signal.iloc[0]))
        
        tsm_qcom_dif.loc[tsm_qcom_dif['Datetime'] == str(last_buy_signal.iloc[0]), 'signal_z_score'] = 0
        #print(tsm_qcom_dif.loc[tsm_qcom_dif['Datetime'] == str(last_sell_signal.iloc[0])])
        print('Delete last BUY signal')

        #print('All buy signals')
        #print(tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == 1])


elif dif_buys_sells < 0:
    last_sell_signal = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == -1].iloc[[-1]]['Datetime']
    #print(last_sell_signal)
    if not last_sell_signal.empty:
        print('Datetime of signal to set to 0 => ', str(last_sell_signal.iloc[0]))
        
        tsm_qcom_dif.loc[tsm_qcom_dif['Datetime'] == str(last_sell_signal.iloc[0]), 'signal_z_score'] = 0
        #print(tsm_qcom_dif.loc[tsm_qcom_dif['Datetime'] == str(last_sell_signal.iloc[0])])
        print('Delete last SELL signal')

        #print('All sell signals')
        #print(tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == -1])



Count buys =>  19
Count sells =>  20
Dif buys/sells =>  -1
Datetime of signal to set to 0 =>  2024-06-04 09:30:00-04:00
Delete last SELL signal


## Calculate Profit using Z-score

In [23]:
# Plot the signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif['Close'], name='TSM-QCOM'))
#fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=10).mean()['Close'], name='10 Ticks MA'))
#fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=50).mean()['Close'], name='50 Ticks MA'))


buy_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == 1]
sell_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == -1]

fig.add_trace(go.Scatter(x=buy_signals_z_score.index, y=buy_signals_z_score['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals_z_score.index, y=sell_signals_z_score['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

# 1 buy || - 1 sell

tsm_data.reset_index(inplace=True)
qcom_data.reset_index(inplace=True)

tsm_data['signal_z_score'] = tsm_qcom_dif['signal_z_score']
qcom_data['signal_z_score'] = - tsm_qcom_dif['signal_z_score'] # opposite signal

fig.show()

# plot the profit and loss

tsm_data['buy_price'] = np.where(tsm_data['signal_z_score'] == 1, tsm_data['Close'], np.nan)
tsm_data['sell_price'] = np.where(tsm_data['signal_z_score'] == -1, tsm_data['Close'], np.nan)

qcom_data['buy_price'] = np.where(qcom_data['signal_z_score'] == 1, qcom_data['Close_NORM'], np.nan)
qcom_data['sell_price'] = np.where(qcom_data['signal_z_score'] == -1, qcom_data['Close_NORM'], np.nan)

tsm_profit = tsm_data['sell_price'].sum() - tsm_data['buy_price'].sum()
qcom_profit = qcom_data['sell_price'].sum() - qcom_data['buy_price'].sum()

print('TSMC Profit => ', tsm_profit)
print('QCOM Profit => ', qcom_profit)

pair_tsm_qcom_profit = tsm_profit + qcom_profit

print('Pair profit => ', pair_tsm_qcom_profit)


TSMC Profit =>  -21.565093994140625
QCOM Profit =>  138.34193200000118
Pair profit =>  116.77683800586055


In [24]:
#Testar Métodos de Mercados Diferentes, Mercados Tecnológicos
# A cointegração muda com 